# Retrieval-augmented generation (RAG)


In [1]:
# !pip install langchain==0.0.331 langchainhub 

In [2]:
# !pip install openai==0.28.1

In [3]:
# !pip install faiss-cpu tiktoken

In [4]:
# !pip install python-dotenv 

In [5]:
# !pip install scikit-learn

In [6]:
# Set env var OPENAI_API_KEY or load from a .env file
import dotenv

dotenv.load_dotenv()

True

In [7]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [8]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(loader.load())

In [9]:
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# vectorstore.save_local('vector.db')
# vectorstore=FAISS.load_local('vector.db', embeddings=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [10]:
# from langchain.retrievers import SVMRetriever
# question="What are key facts from the articles?"
# svm_retriever = SVMRetriever.from_documents(splits, OpenAIEmbeddings())
# docs_svm = svm_retriever.get_relevant_documents(question)
# len(docs_svm)

In [11]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | rag_prompt_custom | llm
)

In [12]:
print(rag_chain.invoke("What are key facts from the articles?").content)

1. Building agents with LLM (large language model) as its core controller is a cool concept.
2. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer, and BabyAGI, serve as inspiring examples.
3. LLM has the potential to be a powerful general problem solver.
4. In a LLM-powered autonomous agent system, LLM functions as the agent's brain, complemented by several key components.
5. The agent system overview includes components such as internet access for searches and information gathering, long-term memory management, GPT-3.5 powered agents for delegation of simple tasks, and file output.


In [13]:
print(rag_chain.invoke("What is the publish date of the article?").content)

The publish date of the article is June 2023.


In [14]:
# from langchain.llms import GPT4All
# local_path = (
#     "./models/ggml-gpt4all-l13b-snoozy.bin"  # replace with your desired local file path
# )

# # Callbacks support token-wise streaming
# callbacks = [StreamingStdOutCallbackHandler()]

# # Verbose is required to pass to the callback manager
# llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)